<a href="https://colab.research.google.com/github/bonvech/MSU-AI/blob/main/EX09_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вспомогательный код

Чтобы результаты экспериментов воспроизводились, зафиксируем seed's:

In [ ]:
import torch
import random
import numpy as np


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)

Для выполнения задания рекомендуется использовать среду с аппаратным ускорителем GPU.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Задание 1. Прогнозирование временного ряда

Используя код из лекции, попробуйте обучить GRU с размерностью скрытого состояния `hidden_size=20` на следующих периодичных данных и визуализируйте результат.

Импорт необходимых библиотек:

In [ ]:
import os
import math
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression

Генерация данных для прогнозирования:

In [ ]:
data = []
total_length = 1000
for i in range(total_length):
    data.append(
        math.sin(i / 2) + math.cos((i) / 6) + i / 100 + (random.random() - 0.5) / 2
    )

dataset = pd.DataFrame(data={"timestamps": range(total_length), "values": data})
dataset.head()

Выведем временной ряд:

In [ ]:
def simple_display(data, xticks, label=None):
    plt.figure(figsize=(12, 4))
    plt.plot(xticks, data, label=label)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
data = dataset["values"]
simple_display(data=data, xticks=dataset["timestamps"], label="Initial data")

Пользуясь примером прогнозирования временного ряда с помощью рекуррентной нейронной сети из лекции, постройте прогнозирующую модель.

Шаги, которые необходимо совершить:
1. Разбиение на train-val-test.
2. Предобработка данных.
3. Создание и обучение модели.
4. Получение предсказаний и расчет метрик качества.

In [ ]:
# Your code here

## Формат результата

1. Графики предсказания модели:
- для обучающих и валидационных данных в режиме "forced prediction",
- для тестовых данных в режиме "rolling prediction".

Пример графиков:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/Exercises/EX09/result_1_1_task_ex09.png" width="800">

2. Оценка качества предсказаний, полученных в режимах, описанных выше, для обучающих, валидационных и тестовых данных.

3. Графики предсказания модели для обучающих, валидационных и тестовых данных в режиме "rolling prediction". Для тестовых данных получить предсказание, как минимум, еще на одну длину тестовых данных.

Пример графиков:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/Exercises/EX09/result_2_1_task_ex09.png" width="800">

# Задание 2. Генерация фамилий

Возьмите следующий набор данных и, используя код из лекции, создайте рекуррентную сеть для генерации фамилий. Подумайте, как получить разные фамилии, начинающиеся на одну и ту же букву.

Импорт необходимых библиотек:

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pprint import pprint

Загрузка данных:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/surnames.txt

In [ ]:
with open("surnames.txt", encoding="utf-8") as s_file:
    surnames_list = [line.strip().lower() for line in s_file.readlines()]

In [ ]:
print(f"Total number of surnames: {len(surnames_list)}")
print("First 10 samples:")
pprint(surnames_list[:10])

In [ ]:
# Your code here

## Формат результата

Модель, генерирующая фамилии по первой букве.

Пример:

а — Аркова

б — Банова

в — Варенков

г — Гаранков

# Задание 3. Прогнозирование многомерного временного ряда

Попробуйте свои силы в анализе многомерных данных. Особенностью таких задач является то, что признаки не являются независимыми и разумно анализировать их одновременно.

В качестве датасета предлагаем [курс биткоина 🛠️[doc]](https://finance.yahoo.com/quote/BTC-USD/history?period1=1410912000&period2=1642118400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true).

Рекомендуем использовать модель LSTM.

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q lightning
!pip install -q pmdarima

In [ ]:
import os
import math
import torch
import random
import datetime
import numpy as np
import pandas as pd
import lightning as L
import torch.nn as nn
import matplotlib.pyplot as plt

from datetime import timedelta
from pmdarima.arima import auto_arima
from lightning.pytorch import Trainer
from torchmetrics import MetricCollection
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from torchmetrics.regression import MeanSquaredError
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

##Загрузка и изучение данных

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/BTC-USD.csv

In [ ]:
dataset = pd.read_csv("BTC-USD.csv", index_col="Date", parse_dates=True)
dataset.drop(columns=["Adj Close"], inplace=True)
dataset.head(5)

У нас есть ежедневные исторические данные о ценах:

* `Open` — цена открытия,
* `High` — верхняя цена,
* `Low` — нижняя цена,
* `Volume` — объём торгов.

Наша цель — взять некоторую последовательность из четырех признаков (скажем, за 100 предыдущих дней) и спрогнозировать целевую переменную `Close` на следующие 50 дней.

Визуализируем целевую переменную.

In [ ]:
plt.rcParams["figure.figsize"] = (12, 3)

plt.plot(dataset["Close"])
plt.xlabel("Time")
plt.ylabel("Price (USD)")
plt.title("Bitcoin price over time")
plt.show()

Посмотрим, какие тут представлены признаки, на основе которых будем предсказывать цену.

In [ ]:
dataset.plot()
plt.show()

Видим единственное значение-всплеск в ряде `Volume`. Из-за такой разницы в масштабе даже толком не видно ничего. Заменим это значение на соседнее.

In [ ]:
idx_max = dataset["Volume"].idxmax()

In [ ]:
prev_value = dataset[dataset.index < idx_max].tail(1)["Volume"][0]
dataset = dataset.replace(dataset["Volume"][idx_max], prev_value)

Визуализируем оставшиеся признаки. Здесь отчётливо видно, что поведение ряда после 2021 года качественно поменялось.

In [ ]:
# Your code here

Сейчас мы исследуем левую часть ряда, до 2021. И попробуем предсказать поведение цены на несколько месяцев вперёд несколькими способами, начиная с быстрых прикидок и заканчивая рекуррентной моделью.

In [ ]:
import datetime
from pmdarima.arima import auto_arima

train_start = datetime.datetime(2018, 1, 1)
train_end = datetime.datetime(2019, 12, 31)

val_start = datetime.datetime(2020, 1, 1)
val_end = datetime.datetime(2020, 6, 1)

test_start = datetime.datetime(2020, 6, 2)
test_end = datetime.datetime(2020, 12, 31)

In [ ]:
train_data = dataset.query("(`Date` >= @train_start) & (`Date` <= @train_end)")
val_data = dataset.query("(`Date` >= @val_start) & (`Date` <= @val_end)")
test_data = dataset.query("(`Date` >= @test_start) & (`Date` <= @test_end)")

##SARIMA

Начнём с того, что попробуем по целевому показателю предсказать самого себя, то есть, как и раньше, работаем с одномерным временным рядом.

In [ ]:
train_data = train_data["Close"]
val_data = val_data["Close"]
test_data = test_data["Close"]

Сразу ограничим количество итераций алгоритма оптимизации модели значением 20 для экономиии времени. В реальной задаче, разумеется, имеет смысл поставить число побольше.

Стартовые параметры взяты по умолчанию.

In [ ]:
stepwise_model = auto_arima(
    train_data,
    start_p=1,
    start_q=1,
    max_p=3,
    max_q=3,
    m=12,
    start_P=0,
    seasonal=True,
    d=1,
    D=1,
    trace=True,
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
    maxiter=20,  # default 50
)
print(stepwise_model.aic())

In [ ]:
print(stepwise_model.summary())

Предскажите вперёд на промежутке валидационных и тестовых данных.

In [ ]:
start_data = # Your code here

future_forecast = stepwise_model.predict(start=start_data, n_periods=# Your code here)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(train_data, label="Train")
plt.plot(val_data, label="Val")
plt.plot(test_data, label="Test")
plt.plot(future_forecast, color="blue", label="Predicted")

plt.title("ARIMA with optimal parameters")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(val_data, future_forecast[: len(val_data)]))
print("RMSE: " + str(rmse))

➕ Baseline. Можно предсказывать на сколь угодно далеко в будущее.

➖ Получилось не очень качественно.

##SARIMAX

Попробуем воспользоваться оставшимися признаками, чтобы уточнить предсказание.

Теперь мы подаём не только `Y`, но и `X`, то есть решаем регрессионную задачу.

In [ ]:
train_data = dataset.query("(`Date` >= @train_start) & (`Date` <= @train_end)")
val_data = dataset.query("(`Date` >= @val_start) & (`Date` <= @val_end)")
test_data = dataset.query("(`Date` >= @test_start) & (`Date` <= @test_end)")

In [ ]:
stepwise_model = auto_arima(
    y=train_data["Close"],
    X=train_data.drop(columns=["Close"]),
    start_p=1,
    start_q=1,
    max_p=3,
    max_q=3,
    m=12,
    start_P=0,
    seasonal=True,
    d=1,
    D=1,
    trace=True,
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
    maxiter=20,  # default 50
)

Для предсказания в функцию `predict` теперь нужно подать признаки.

In [ ]:
start_data = # Your code here
future_forecast_val = stepwise_model.predict(X = # Your code here, start=start_data, n_periods=# Your code here)

start_data = # Your code here
future_forecast_test = stepwise_model.predict(X = # Your code here, start=start_data, n_periods=# Your code here)

Здесь есть особенность в том, что начальные индексы по времени `future_forecast_val` и `future_forecast_test` совпадут. Поэтому для визуализации мы сдвинем на размер валидации, т.е. на 153 дня.

In [ ]:
future_forecast_test.index = future_forecast_test.index.shift(153, freq="D")

Проверим, что даты сдвинулись:

In [ ]:
future_forecast_test

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(train_data["Close"], label="Train")
plt.plot(val_data["Close"], label="Val")
plt.plot(test_data["Close"], label="Test")
plt.plot(future_forecast_val, label="Predicted Val")
plt.plot(future_forecast_test, label="Predicted Test")


plt.title("ARIMA with optimal parameters")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(val_data["Close"], future_forecast_val))
print("RMSE: " + str(rmse))

Ошибка уменьшилась примерно вдвое.

Фишка SARIMAX в том, что мы можем придумывать дополнительные признаки и подавать их в ту же модель, просто добавляя столбцы в таблице в `pandas`.

Придумайте какой-нибудь признак, который хотя бы немного уменьшит ошибку на валидации.

In [ ]:
# Your code here

Обучите модель на расширенном датасете.

In [ ]:
stepwise_model = auto_arima(
    y=train_data["Close"],
    X=train_data.drop(columns=["Close"]),
    start_p=1,
    start_q=1,
    max_p=3,
    max_q=3,
    m=12,
    start_P=0,
    seasonal=True,
    d=1,
    D=1,
    trace=True,
    error_action="ignore",
    suppress_warnings=True,
    stepwise=True,
    maxiter=20,  # default 50
)

Предскажите вперёд. Не забудьте подправить даты на тесте.

In [ ]:
# Your code here

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(train_data["Close"], label="Train")
plt.plot(val_data["Close"], label="Val")
plt.plot(test_data["Close"], label="Test")
plt.plot(future_forecast_val, label="Predicted Val")
plt.plot(future_forecast_test, label="Predicted Test")


plt.title("ARIMA with optimal parameters")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
rmse = np.sqrt(mean_squared_error(val_data["Close"], future_forecast_val))
print("RMSE: " + str(rmse))

Мы смогли улучшить качество путём конструирования признаков.

С помощью относительно простой линейной модели мы можем получить приемлемое качество. Если уделить ещё больше времени конструированию признаков или дать больше итераций подбора модели, качество будет ещё выше. Возможно, конкретно ваша задача на этом этапе уже может считаться решённой. Не беритесь сразу за обучение нелинейных моделей.

➕ Получилось довольно качественно.

➖ Не можем предсказывать будущие значения. Решение регрессионной задачи есть только там, где уже есть признаки.

##Нелинейная модель

❗ Экспериментируя с Auto-ARIMA, вы могли к этому моменту заполнить бОльшую часть оперативной памяти. Если это так, перезагрузите среду.

💥 Основная цель этой части – дать вам практику работы со сложным многомерными данными из реального мира. При неудачных параметрах качество предсказания модели может оказаться хуже, чем у SARIMAX. Если модель сможет верно оценить поведение данных в будущем, этого уже достаточно. Дальнейшее улучшение – вопрос перебора параметров и машинного времени.

Теперь воспользуемся нелинейной моделью. Адаптируем код из лекции для работы с многомерными данными.

Загрузим заново данные и поделим на подвыборки.

In [ ]:
dataset = pd.read_csv("BTC-USD.csv", index_col="Date", parse_dates=True)
dataset.drop(columns=["Adj Close"], inplace=True)
dataset.head(5)

### Разбиение на train-val-test

Здесь мы дополнительно вынуждены отрезать часть начальных данных, ибо такое поведение ряда далее не наблюдается.

In [ ]:
import datetime

# train_start = datetime.datetime(2018, 1, 1)
train_start = datetime.datetime(2019, 1, 1)
train_end = datetime.datetime(2019, 12, 31)

val_start = datetime.datetime(2020, 1, 1)
val_end = datetime.datetime(2020, 6, 1)

test_start = datetime.datetime(2020, 6, 2)
test_end = datetime.datetime(2020, 12, 31)

In [ ]:
train_data = dataset.query("(`Date` >= @train_start) & (`Date` <= @train_end)")
val_data = dataset.query("(`Date` >= @val_start) & (`Date` <= @val_end)")
test_data = dataset.query("(`Date` >= @test_start) & (`Date` <= @test_end)")

In [ ]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

Для каждой части данных (train, val, и test) запишем в словарь временные метки и исходные данные.

In [ ]:
split = {"train": {}, "val": {}, "test": {}}

for part, data_part in zip(split, [train_data, val_data, test_data]):
    split[part]["timestamps"] = data_part.index
    split[part]["data"] = data_part.values  # Close is #3 column

Проверим размеры:

In [ ]:
print(split["test"]["data"].shape)
print(split["val"]["data"].shape)
print(split["train"]["data"].shape)

Отобразим разделенные данные:

In [ ]:
def initial_data_display(split):
    plt.figure(figsize=(12, 4))
    for part in split:
        plt.plot(split[part]["timestamps"], split[part]["data"][:, 3], label=part)
    plt.title("Initial data")
    plt.legend()
    plt.grid()
    plt.show()


initial_data_display(split)

###Устранение тренда

In [ ]:
from sklearn.linear_model import LinearRegression


class TimeSeriesTransform:
    def __init__(self, apply_log=False):
        self.slope = None
        self.apply_log = apply_log

    def fit(self, train_data: np.ndarray):
        data = train_data
        if self.apply_log:
            data = np.log(data + 1)  # to avoid log(0)

        x = np.arange(len(data))
        x_centered = x - x.mean()

        data_centered = data - data.mean(axis=0)
        # print (data_centered.shape)

        slopes = []
        for i in range(data_centered.shape[1]):
            reg = LinearRegression(fit_intercept=False).fit(
                x_centered.reshape(-1, 1), data_centered[:, i].reshape(-1, 1)
            )
            slopes.append(reg.coef_[0])

        self.slope = slopes

        return self

    def transform(self, data: np.ndarray, window_size: int):
        if self.slope is None:
            raise ValueError("call fit before transform")

        if self.apply_log:
            data = np.log(data + 1)  # to avoid log(0)

        x = np.arange(len(data))
        x_centered = x - x.mean()
        trend = self.slope * x_centered

        anchor_value = data[window_size]
        data_centered = data - data.mean(axis=0)
        data_detrended = data_centered - trend.T
        return anchor_value, data_detrended

    def inverse_transform(self, anchor_value: float, data_detrended: np.ndarray):
        if self.slope is None:
            raise ValueError("call fit before inverse_transform")

        x = np.arange(len(data_detrended))
        x_centered = x - x.mean()
        trend = self.slope * x_centered

        trend = trend.T
        data = np.squeeze(data_detrended) + trend

        data = (
            data
            - np.array([data[0, :]] * len(data))
            + np.array([anchor_value] * len(data))
        )
        if self.apply_log:
            data = np.exp(data) - 1
        return data

Сохраним опорные точки (`anchor_value`) для каждой из подвыборок, чтобы знать, в каком масштабе нужно добавлять тренд в дальнейшем при вызове `inverse_transform`.

In [ ]:
window_size = # Your code here

# Your code here

Отобразим преобразованные данные, на которых теперь можно обучать, валидировать и тестировать нейронную сеть:

In [ ]:
def transformed_data_display(split):
    for i in range(5):
        plt.figure(figsize=(12, 4))
        for part in split:
            plt.plot(
                split[part]["timestamps"],
                split[part]["data_transformed"][:, i],
                label=part,
            )
        plt.title("Transformed data")
        plt.legend()
        plt.grid()
        plt.show()


transformed_data_display(split)

###Создание датасета

Создадим датасет: будем обучать нейронную сеть по последовательности из `seq_len` элементов предсказывать `seq_len + 1`-й.

In [ ]:
from torch.utils.data import Dataset

# Your code here

Для каждой части данных создадим `DataSet` и `DataLoader`:

In [ ]:
from torch.utils.data import DataLoader

# Your code here

Проверим размеры и количество батчей во всех подвыборках:

In [ ]:
# Your code here

###Создание модели

Рекомендуем использовать модель LSTM.

Здесь вы сможете попробовать сделать вашу модель более мощной, так как данные усложнились. Учтите, что теперь у вас многомерный ряд, а значит размеры входа и выхода изменились.

In [ ]:
import torch.nn as nn


# Your code here

Проверим, что входные данные одного батча проходят через модель.

In [ ]:
# Your code here

###Получение предсказаний

In [ ]:
def forced_predict(model, split, part):
    y_true = []
    y_pred = []
    model.eval()
    dataset = split[part]["dataset"]
    with torch.no_grad():
        for x, y in dataset:
            out = model(x)
            y_true.append(y.tolist())
            y_pred.append(out.tolist())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    transform = split["train"]["transform"]
    y_true = transform.inverse_transform(split[part]["anchor_value"], y_true)
    y_pred = transform.inverse_transform(split[part]["anchor_value"], y_pred)

    return y_true, y_pred

In [ ]:
def rolling_predict(model, split, part, forecast_horizon):
    y_pred = []
    model.eval()
    dataset = split[part]["dataset"]
    x, _ = dataset[0]

    with torch.no_grad():
        for _ in range(forecast_horizon):
            out = model(x).view(-1, 1)  # for concatenation shape compatibility
            y_pred.append(out.tolist())
            # drop first element and add new prediction
            x = torch.cat([x[1:, :], out.T], dim=0)
    y_pred = np.array(y_pred)

    transform = split["train"]["transform"]
    y_pred = transform.inverse_transform(split[part]["anchor_value"], y_pred)

    return y_pred

Для проверки получим предсказания необученной модели: в режиме "forced prediction" для обучающих и валидационных данных и в режиме "rolling prediction" для тестовых.

In [ ]:
# Your code here

Отобразим актуальные и предсказанные данные.

In [ ]:
from datetime import timedelta


def display_pred_with_rolling_test(
    split, show_only_target=False, post_test=False, dataset=dataset
):
    for i in range(5):
        plt.figure(figsize=(12, 4))
        if show_only_target:
            i = 3  # show Close column
        for part in split:
            timestamps = split[part]["timestamps"][window_size:]
            real_data = split[part]["y_true"][:, i]
            pred_data = split[part]["y_pred"][:, i]

            if part in ("train", "val"):

                plt.plot(timestamps, real_data, label=f"{part}/real")
                plt.plot(timestamps, pred_data, label=f"{part}/predicted")

            if part == "test":
                plt.plot(timestamps, real_data, label=f"{part}/real")
                # difference from the lecture code
                future_timestamps = pd.date_range(
                    timestamps[0],
                    timestamps[0] + timedelta(days=(len(pred_data)) - 1),
                    freq="D",
                )
                plt.plot(future_timestamps, pred_data, label=f"{part}/predicted")

        # difference from the lecture code
        if post_test:
            future_timestamps = pd.date_range(
                timestamps[-1],
                timestamps[-1] + timedelta(days=(len(pred_data))),
                freq="D",
            )
            furure_data = dataset["Close"][
                timestamps[-1] : timestamps[-1] + timedelta(days=(len(pred_data)))
            ]
            plt.plot(future_timestamps, furure_data, label="real future data")

        plt.title("Real vs Predicted")
        plt.legend()
        plt.grid()
        plt.show()
        if show_only_target:
            return  # show Close column


display_pred_with_rolling_test(split)

Оценим ошибку RMSE для предсказаний необученной модели (только для целевой переменной).

In [ ]:
# Your code here

###Обучение

In [ ]:
import lightning as L
from torchmetrics import MetricCollection
from torchmetrics.regression import MeanSquaredError

class TimeSeriesPipeline(L.LightningModule):
    def __init__(
        self,
        model,
        exp_name="baseline",
        criterion=nn.MSELoss(),
        optimizer_class=,# Your code here
    ):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.optimizer_class = optimizer_class
        metrics = MetricCollection([MeanSquaredError()])
        self.train_metrics = metrics.clone(postfix="/train")
        self.valid_metrics = metrics.clone(postfix="/val")

    def configure_optimizers(self):
        optimizer = self.optimizer_class(
            self.model.parameters()
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.log("Loss/train", loss, prog_bar=True)
        self.train_metrics.update(out, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y = torch.squeeze(y)
        out = self.model(x)
        loss = self.criterion(out, y)
        self.log("Loss/val", loss, prog_bar=True)
        self.valid_metrics.update(out, y)

    def on_training_epoch_end(self):
        train_metrics = self.train_metrics.compute()
        self.log_dict(train_metrics)
        self.train_metrics.reset()

    def on_validation_epoch_end(self):
        valid_metrics = self.valid_metrics.compute()
        self.log_dict(valid_metrics)
        self.valid_metrics.reset()

Создадим пайплайн и запустим обучение с сохранением лучшей модели по минимальному MSE на валидационной выборке.

In [ ]:
# Your code here

#### Восстановление модели из контрольной точки

In [ ]:
# Your code here

## Предсказания обученной модели

Получим и отобразим предсказания обученной модели: в режиме "forced prediction" для обучающих и валидационных данных и в режиме "rolling prediction" для тестовых.

In [ ]:
# Your code here

display_pred_with_rolling_test(split, show_only_target=True)

Посчитайте RMSE для целевой переменной.

In [ ]:
# Your code here

На валидации мы добились меньшей ошибки, чем была с SARIMAX. Однако в режиме "rolling prediction" картина не такая хорошая. Однако то, что верно предсказано резкое повышение ряда, является хорошим результатом.

Также мы можем получить предсказания в режиме "rolling prediction" и для обучающих и валидационных данных. Для примера также получим предсказания на тестовых данных.

In [ ]:
# Your code here

display_pred_with_rolling_test(
    split, show_only_target=True, post_test=True, dataset=dataset
)

Если вы добились верной оценки поведения модели в будущем, это можно считать достаточным результатом.

Для того, чтобы улучшить качество предсказания и приблизиться к качеству, как в первом задании, можно попробовать сделать более сложную модель (размер вектора hidden в рекуррентном слое, количество таких слоёв, дополнительные линейные слои, тип оптимизатора).

Главная сложность: сами данные, которые меняют своё поведение во времени. Причём иногда очень сильно и внезапно.

➕ Можем предсказывать сколь угодно далеко. Качество лучше, чем у SARIMA.

➖ Долго, муторно, можем вовсе не подобрать подходящую модель.

**Выводы:**

1. SARIMA используется в качестве baseline.
2. SARIMAX предсказывает неплохо, но только там, где есть признаки. Для предсказания дальше нужно снова возвращаться к SARIMA (или реализовывать цикл, SARIMA будет каждый раз переобучаться на новые, предсказанные данные).
3. Нейросеть работает на любом промежутке времени, но сложность разработки может быть крайне высокой.

## Формат результата

Графики предсказаний, RMSE для оценки качества предсказания.

# Задание 4*. Посимвольная генерация текста

Возьмите произведение Гете "Фауст" и обучите на нем LSTM-модель для посимвольной генерации текста. Вместо one-hot кодирования используйте `nn.Embedding` [🛠️[doc]](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html). При обучении игнорируйте знаки препинания и номера страниц.

[[doc] 🛠️ Word Embeddings Tutorial](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)


Импорт необходимых библиотек:

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

Загрузка данных:

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/Faust.txt

In [ ]:
with open("Faust.txt") as text_file:
    faust_text = "".join(text_file.readlines())

In [ ]:
# Your code here

## Формат результата

Сгенерерированный текст

Пример текста:

"все все от бесстыдные старой

все в нем получше все стремленья

поддержки с собой в сердце воздух своей

и в вечной страсти восстанет свой предлог

привет вам слуга в сладком страшней стране

и в мире все вражда станет станет

в поле на пользу своим воспоминанья"
